Based on these posts: 
* https://medium.com/@chrisfotache/text-classification-in-python-pipelines-nlp-nltk-tf-idf-xgboost-and-more-b83451a327e0
* https://github.com/keisukeirie/Amazon_review_helpfulness_prediction
* https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/
* https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22

In [ ]:
!pip install -q boto3
!pip install -q scikit-learn==0.20.3
!pip install -q nltk==3.4.5

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

### Create Scikit-Learn Pipeline with SGDClassifier

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier

pipeline = Pipeline([
    ('features', FeatureUnion([
        ('body', Pipeline([
            ('body_text_selector', TextSelector('review_body')),
            ('tfidf_vectorizer', TfidfVectorizer(tokenizer=Tokenizer, stop_words="english",
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
    ])),
    ('classifier', SGDClassifier(learning_rate='optimal'))
])

Fit the model

In [ ]:
pipeline.fit(X_train, y_train)

Predict and calculate metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

test_preds = pipeline.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, test_preds))
print('Precision: ', precision_score(y_test, test_preds, average=None))

In [ ]:
print(classification_report(y_test, test_preds))

In [ ]:
df_cm = confusion_matrix(y_test, test_preds)
df_cm

In [ ]:
# TODO:  Explain the classifier

# Test Metrics

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#plt.figure(figsize = (10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

# TODO:  Balance the dataset before training
# TODO:  Add labels to each quadrant (predicted y-axis, actual x-axis)

plt.show()